In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import find_peaks
from scipy.spatial.distance import cosine, euclidean
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import cv2
from PIL import Image
from evaluation.neuroprobe.config import ROOT_DIR, SAMPLING_RATE
from subject.braintreebank import BrainTreebankSubject
import pandas as pd
import math

SUBJECT_TRIAL_TO_MOVIE = {
    (1,0): 'fantastic-mr-fox.mp4',
    (1,1): 'the-martian.mp4',
    (1,2): 'thor-ragnarok.mp4',
    (2,0): 'venom.mp4',
    (2,1): 'spider-man-3-homecoming.mp4',
    (2,2): 'guardians-of-the-galaxy.mp4',
    (2,3): 'guardians-of-the-galaxy-2.mp4',
    (2,4): 'avengers-infinity-war.mp4',
    (2,5): 'black-panther.mp4',
    (2,6): 'aquaman.mp4',
    (3,0): 'cars-2.mp4',
    (3,1): 'lotr-1.mp4',
    (3,2): 'lotr-2.mp4',
    (4,0): 'shrek-the-third.mp4',
    (4,1): 'megamind.mp4',
    (4,2): 'incredibles.mp4',
    (5,0): 'fantastic-mr-fox.mp4',
    (6,0): 'megamind.mp4',
    (6,1): 'toy-story.mp4',
    (6,2): 'coraline.mp4',
    (7,0): 'cars-2.mp4',
    (7,1): 'megamind.mp4',
    (8,0): 'sesame-street-episode-3990.mp4',
    (9,0): 'ant-man.mp4',
    (10,0): 'cars-2.mp4',
    (10,1): 'spider-man-far-from-home.mp4',
}

CLIP_DIR = "/om2/data/public/braintreebank_movies_clip_preprocessed_2/"
MOVIES_DIR = "/om2/data/public/braintreebank_movies/"

In [2]:
def get_movie_data(movie):
    clip_features_path = os.path.join(CLIP_DIR, movie.replace('.mp4', '_clip_features.npy'))
    timestamps_path = os.path.join(CLIP_DIR, movie.replace('.mp4', '_timestamps.npy'))
    movie_path = os.path.join(MOVIES_DIR, movie)

    clip_features = np.load(clip_features_path)  # shape: (num_samples, feature_dim)
    timestamps = np.load(timestamps_path)
    return clip_features, timestamps, movie_path

In [3]:
movie = 'ant-man.mp4'
clip_features, timestamps, movie_path = get_movie_data(movie)
clip_features

array([[-0.013054, -0.023   , -0.0494  , ...,  0.0675  , -0.03183 ,
        -0.0256  ],
       [-0.013054, -0.023   , -0.0494  , ...,  0.0675  , -0.03183 ,
        -0.0256  ],
       [-0.013054, -0.023   , -0.0494  , ...,  0.0675  , -0.03183 ,
        -0.0256  ],
       ...,
       [-0.013054, -0.023   , -0.0494  , ...,  0.0675  , -0.03183 ,
        -0.0256  ],
       [-0.013054, -0.023   , -0.0494  , ...,  0.0675  , -0.03183 ,
        -0.0256  ],
       [-0.013054, -0.023   , -0.0494  , ...,  0.0675  , -0.03183 ,
        -0.0256  ]], dtype=float16)

In [16]:
def detect_scene_changes(features, timestamps, threshold=0.5):
    scene_changes = []
    for i in range(1, len(features)):
        prev_frame = features[i-1]
        curr_frame = features[i]

        similarity = np.dot(prev_frame, curr_frame)
        if similarity < threshold:
            scene_changes.append(timestamps[i])
    
    return scene_changes

scene_change_timestamps = detect_scene_changes(clip_features, timestamps, threshold=0.6)

print(f"Found {len(scene_change_timestamps)} scene changes in {movie}")
print(f"Scene change timestamps: {scene_change_timestamps}")

print(f"\nSummary:")
print(f"Movie: {movie}")
print(f"Total frames: {len(clip_features)}")
print(f"Movie duration: {timestamps[-1]:.2f} seconds")
print(f"Scene changes detected: {len(scene_change_timestamps)}")
print(f"Average time between scene changes: {timestamps[-1]/len(scene_change_timestamps):.2f} seconds" if scene_change_timestamps else "No scene changes detected")


Found 100 scene changes in ant-man.mp4
Scene change timestamps: [6.506506506506507, 13.722055388722056, 30.8641975308642, 128.00300300300302, 292.6676676676677, 529.9883216549883, 551.0093426760094, 689.9816483149817, 695.362028695362, 699.1991991991993, 809.3927260593928, 830.3303303303304, 1056.8902235568903, 1071.3630296963631, 1262.6376376376377, 1425.9259259259259, 1427.2605939272607, 1535.1184517851186, 1608.566900233567, 1625.1251251251251, 1710.4604604604606, 1734.9432766099433, 1850.7674341007676, 2122.288955622289, 2192.0253586920253, 2337.4624624624626, 2872.455789122456, 3097.847847847848, 3099.516182849516, 3145.5205205205207, 3153.2365699032366, 3154.904904904905, 3167.667667667668, 3187.771104437771, 3191.358024691358, 3243.743743743744, 3276.5682349015683, 3447.155488822156, 3507.632632632633, 3529.446112779446, 3738.613613613614, 3750.9592926259593, 3752.544210877544, 3756.9652986319657, 3769.060727394061, 3771.062729396063, 3779.2375709042376, 3780.447113780447, 3783.

In [17]:
def detect_scene_changes_derivative(features, timestamps, threshold=0.1):
    scene_changes = []
    derivatives = []

    for i in range(1, len(features)):
        derivative = np.linalg.norm(features[i] - features[i-1])
        derivatives.append(derivative)

    for i, derivative in enumerate(derivatives):
        if derivative > threshold:
            scene_changes.append(timestamps[i+1])  # i+1 because derivatives start from second frame
    
    return scene_changes

# Test the derivative-based approach
scene_change_timestamps_deriv = detect_scene_changes_derivative(
    clip_features, timestamps, threshold=0.3
)

print(f"Found {len(scene_change_timestamps_deriv)} scene changes using derivatives in {movie}")
print(f"Scene change timestamps: {scene_change_timestamps_deriv}")

print(f"\nSummary (Derivative Method):")
print(f"Movie: {movie}")
print(f"Total frames: {len(clip_features)}")
print(f"Movie duration: {timestamps[-1]:.2f} seconds")
print(f"Scene changes detected: {len(scene_change_timestamps_deriv)}")
print(f"Average time between scene changes: {timestamps[-1]/len(scene_change_timestamps_deriv):.2f} seconds" if scene_change_timestamps_deriv else "No scene changes detected")


Found 10008 scene changes using derivatives in ant-man.mp4
Scene change timestamps: [6.506506506506507, 8.800467133800467, 13.722055388722056, 15.890890890890892, 21.02102102102102, 24.065732399065734, 27.402402402402405, 29.23757090423757, 29.487821154487822, 29.52952952952953, 29.77977977977978, 29.821488154821488, 29.946613279946614, 29.988321654988322, 30.8641975308642, 31.61494828161495, 32.44911578244912, 35.53553553553554, 39.622956289622955, 40.749082415749086, 41.75008341675009, 43.001334668001334, 46.671671671671675, 47.881214547881214, 53.42842842842843, 57.47414080747414, 57.97464130797464, 62.312312312312315, 63.39673006339673, 65.89923256589924, 66.14948281614949, 66.35802469135803, 66.48314981648315, 66.73340006673341, 67.1921921921922, 67.48415081748415, 67.60927594260927, 68.98565231898566, 70.27861194527861, 74.15749082415749, 75.15849182515849, 76.78511845178512, 77.82782782782783, 80.2052052052052, 82.58258258258259, 83.95895895895896, 90.54888221554889, 97.13880547